Nonlinear Shells
===

Joint work with M. Neunteufel

Geometric model and meshing. left surface only needed for boundary conditions.

In [2]:
from netgen.csg import *
from ngsolve import *
import netgen.gui

geo = CSGeometry()
cyl = Cylinder (Pnt(0,0,0),Pnt(1,0,0),0.4).bc("cyl")
left = Plane (Pnt(0,0,0), Vec(-1,0,0)).bc("left")
finitecyl = cyl * left * Plane(Pnt(1,0,0), Vec(1,0,0))

geo.AddSurface (cyl, finitecyl)
geo.AddSurface (left, finitecyl)

mesh = Mesh(geo.GenerateMesh(maxh=0.2))
mesh.Curve(3)
Draw (mesh)

Fields are: displacement $u \in [H^1]^3$, rotation $\beta \in H(\text{curl},S)$, stress $\sigma \in H(\text{div div},S)$

In [4]:
order = 3

fes1 = HDivDivSurface(mesh, order=order-1)
fes2 = H1(mesh, order=order, dirichlet="left")
fes3 = HCurl(mesh, order=order-1, dirichlet="left")
fes = FESpace([fes1,fes2,fes2,fes2,fes3])

sigma,u1,u2,u3,beta = fes.TrialFunction()
tau,v1,v2,v3,delta = fes.TestFunction()

sigma = sigma.Trace()
tau = tau.Trace()
beta = beta.Trace()
delta = delta.Trace()

gradv = CoefficientFunction((grad(v1).Trace(),grad(v2).Trace(),grad(v3).Trace()),dims=(3,3))
gradu = CoefficientFunction((grad(u1).Trace(),grad(u2).Trace(),grad(u3).Trace()),dims=(3,3))

nsurf = specialcf.normal(3)
t = specialcf.tangential(3)

def Cross(a,b):
    return CoefficientFunction( (a[1]*b[2]-a[2]*b[1],a[2]*b[0]-a[0]*b[2],a[0]*b[1]-a[1]*b[0]) )

nel = Cross(nsurf, t)

ngradv = gradv.trans*nsurf
ngradu = gradu.trans*nsurf

sigman = sigma*nel
taun = tau*nel

def tang(u):
    return (u*t)*t

thickness = 0.1

Membrane energy with primal method for displacement vector $u$:
$$
\int_S W(C_{tt}(u)) 
$$
Shear energy, $\beta$ a rotation field in tangential plane:
$$
\int_S | n^T \nabla u - \beta |^2 
$$
Bending energy (with TDNNS mixed method):
$$
t^2 \int_S | \varepsilon_{tt}(\beta) |^2
$$

In [5]:
a = BilinearForm(fes, symmetric=True)
a += SymbolicBFI( (-1/thickness**2*InnerProduct(sigma,tau)).Compile(True), BND)
a += SymbolicBFI( (-div(sigma)*delta - div(tau)*beta).Compile(True), BND )
a += SymbolicBFI( (sigman * tang(delta) + taun*tang(beta)).Compile(True), BND, element_boundary=True )

a += SymbolicEnergy( (0.5*(ngradu-beta)*(ngradu-beta)).Compile(True), BND )

nsurfmat = CoefficientFunction( (nsurf,), dims=(3,1))
Ptau = Id(3) - nsurfmat * nsurfmat.trans
Ftau = gradu + Ptau
Ctautau = Ftau.trans * Ftau
Etautau = Ctautau - Ptau
a += SymbolicEnergy( (InnerProduct (Etautau, Etautau)).Compile(True), BND )

factor = Parameter(0.1)
a += SymbolicEnergy( (-factor*y * u2).Compile(True), BND )

u = GridFunction(fes)

uvec = CoefficientFunction(u.components[1:4])

Draw(u.components[4], mesh, "rotations")
Draw(uvec, mesh, "disp", sd=4)

solsigma = CoefficientFunction([u.components[0]], None)
# Draw(u.components[0], mesh, "sigma")
Draw(solsigma, mesh, "sigma", sd=4)

Increase the load step-wise, solve the non-linear problem by Newton's method. First and second order derivatives are computed by automatic differentiation.

In [6]:
from newtonsolver import NewtonMinimization

for loadstep in range(4):
    print ("loadstep ", loadstep)
    factor.Set (3*(loadstep+1))
    with TaskManager():
        NewtonMinimization (a, u)
    Redraw(blocking=True)

loadstep  0
Newton iteration  0
energy =  9.385554547969788e-32
tau =  0.5
energy uh =  -0.046407695384766294
Newton iteration  1
energy =  -0.046407695384766294
Newton iteration  2
energy =  -0.059698110554143466
Newton iteration  3
energy =  -0.060188892523224465
Newton iteration  4
energy =  -0.06019700858324533
Newton iteration  5
energy =  -0.060197013034633824
Newton iteration  6
energy =  -0.06019701303463584
loadstep  1
Newton iteration  0
energy =  -0.16212877624059607
Newton iteration  1
energy =  -0.18047030359955413
Newton iteration  2
energy =  -0.18814113448289646
Newton iteration  3
energy =  -0.188292787973254
Newton iteration  4
energy =  -0.18829323589939165
Newton iteration  5
energy =  -0.1882932359053115
loadstep  2
Newton iteration  0
energy =  -0.3391246325802083
Newton iteration  1
energy =  -0.35472110553465824
Newton iteration  2
energy =  -0.3580222028628129
Newton iteration  3
energy =  -0.35820310983034076
Newton iteration  4
energy =  -0.35823429355522934
